# Docker Code Exectution
https://microsoft.github.io/autogen/docs/tutorial/code-executors#docker-execution

In [20]:
import tempfile
import autogen

from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

## Setting up the Agent

In [21]:


# Create a temporary directory to store the code files.
#temp_dir = tempfile.TemporaryDirectory()

# Create a Docker command line code executor.
executor = DockerCommandLineCodeExecutor(
    image="python:3.13.0b4-slim",  # Execute code using the given docker image name.
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir="coding",  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration that uses docker.
code_executor_agent_using_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the docker command line code executor.
    human_input_mode="TERMINATE",  # Always take human input for this agent for safety.
)

## Use it in Conversation

In [22]:
# Set your API endpoints
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4o"],
    },
)

gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

In [23]:
# The code writer agent's system message is to instruct the LLM on how to use
# the code executor in the code executor agent.
code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""

code_writer_agent = ConversableAgent(
    "code_writer_agent",
    system_message=code_writer_system_message,
    llm_config=gpt4_config,
    code_execution_config=False,  # Turn off code execution for this agent.
)

In [24]:
code_executor_agent_using_docker.initiate_chat(
    code_writer_agent,
    message="Write Python code to calculate the 14th Fibonacci number.",
)



code_executor_agent_docker (to code_writer_agent):

Write Python code to calculate the 14th Fibonacci number.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer_agent (to code_executor_agent_docker):

Sure, I will provide you with a Python script to calculate the 14th Fibonacci number.

```python
# filename: fibonacci_14.py

def fibonacci(n):
    if n <= 0:
        return "Input should be a positive integer"
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b

# Calculate the 14th Fibonacci number
fib_14 = fibonacci(14)
print(f"The 14th Fibonacci number is: {fib_14}")
```

Save the code in a file named `fibonacci_14.py` and execute it. The script will print the 14th Fibonacci number.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...



ChatResult(chat_id=None, chat_history=[{'content': 'Write Python code to calculate the 14th Fibonacci number.', 'role': 'assistant'}, {'content': 'Sure, I will provide you with a Python script to calculate the 14th Fibonacci number.\n\n```python\n# filename: fibonacci_14.py\n\ndef fibonacci(n):\n    if n <= 0:\n        return "Input should be a positive integer"\n    elif n == 1:\n        return 0\n    elif n == 2:\n        return 1\n    else:\n        a, b = 0, 1\n        for _ in range(2, n):\n            a, b = b, a + b\n        return b\n\n# Calculate the 14th Fibonacci number\nfib_14 = fibonacci(14)\nprint(f"The 14th Fibonacci number is: {fib_14}")\n```\n\nSave the code in a file named `fibonacci_14.py` and execute it. The script will print the 14th Fibonacci number.', 'role': 'user'}, {'content': 'exitcode: 0 (execution succeeded)\nCode output: The 14th Fibonacci number is: 233\n', 'role': 'assistant'}, {'content': 'Great! The code executed successfully and calculated the 14th Fi

In [25]:
# When the code executor is no longer used, stop it to release the resources.
executor.stop()